In [2]:
import cv2
from matplotlib import pyplot as plt
from ipywidgets import interact, interactive, fixed, interact_manual
import ipywidgets as widgets
import numpy as np
from PIL import Image
from PIL import ImageFilter
from matplotlib import animation
from PIL import ImageEnhance

import os

%matplotlib inline

# for loading array 

from scipy.io import loadmat
import pandas as pd
import numpy as np
import mat4py

In [3]:
data =  pd.read_csv(r'C:\Users\nkhefif\Desktop\WhiskiTestsVideo\26_02_test_\Mouse24_2020-02-18T17.26.39_processed_MJPG.measurements.csv',encoding='latin1',sep = ",", header = 0)
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
display(data)

#test_row = data.iloc[[1]]
#print([int(test_row.follicle_x)]) #test_row.tip_x(0))

,fid,wid,label,face_x,face_y,length,score,angle,curvature,follicle_x,follicle_y,tip_x,tip_y
0,0,0,0,602,1077,358.611481,420.490326,0.448737,0.000210,588.084229,500.360107,937.909485,463.591614
1,0,1,0,602,1077,115.062782,211.384583,-89.214172,0.006537,213.781281,717.961426,270.242462,622.242493
2,0,2,0,602,1077,137.502014,234.832642,-166.370758,-0.004346,238.057068,139.862228,116.000000,91.033409
3,0,3,0,602,1077,508.009186,353.213928,-98.969452,0.000794,631.118164,675.984436,603.904663,177.995834
4,0,4,0,602,1077,106.122200,273.630920,141.197723,0.007948,354.805267,33.694336,255.107254,24.705307
...,...,...,...,...,...,...,...,...,...,...,...,...,...
200740,497,514,0,602,1077,20.917616,170.055328,-47.270462,-0.012260,438.038696,592.029663,449.769623,574.867004
200741,497,515,0,602,1077,260.713593,293.874054,-135.048126,0.001865,962.048706,243.936493,891.268677,2.902200
200742,497,516,0,602,1077,83.431389,236.297379,-38.678429,0.002824,220.851746,79.823318,281.884888,24.819315
200743,497,517,0,602,1077,429.010223,398.088928,153.317093,0.005092,916.042175,283.679596,662.056580,61.970562


In [8]:
#Fonction Resize

def calculateNewSize(width,height,WishForNEwWidth):
    ratio = WishForNEwWidth/width
    Nheight = int(height * ratio)
    Nwidth = int(width * ratio)    
    return Nwidth , Nheight



HandleBEHAV = cv2.VideoCapture(r'C:\Users\nkhefif\Desktop\WhiskiTestsVideo\26_02_test_\Mouse24_2020-02-18T17.26.39_processed_MJPG.avi')
HandleVSD = cv2.VideoCapture(r'C:\Users\Nicolas\Desktop\Mouse17_Correl_BVH_VSD_test\Session4_trial4\S04-T04_2.mov')

length = int(HandleBEHAV.get(cv2.CAP_PROP_FRAME_COUNT))

_ , IMG0 = HandleBEHAV.read() #Lecture de la première frame de compormement



@interact

def show_image(time=widgets.IntSlider(min=0,max=length,step=1,value=200), \
Brightness=widgets.FloatSlider(min=0,max=4,step=0.015,value=1), \
binarization=widgets.FloatSlider(min=-1,max=2,step=0.01,value=-1), \
Denoise = False, VSD=False, substract= False, EnhanceEdge=False, EnhanceEdgeMore=False ):
    

    HandleBEHAV.set(cv2.CAP_PROP_POS_FRAMES, time)
    _ , BEHi = HandleBEHAV.read()
     
    
    #HandleVSD.set(cv2.CAP_PROP_POS_FRAMES, time)
    #_ , VSDi = HandleVSD.read() 
    #VSDi = cv2.cvtColor(VSDi, cv2.COLOR_BGR2RGB)
   
    
    if Denoise == True:
        
        BEHi = cv2.fastNlMeansDenoisingColored(BEHi) 
        
        #autres manière (marchent pas forcement, je garde au cas ou)
        #gray = cv2.cvtColor(BEHi, cv2.COLOR_RGB2GRAY)
        
        #BEHi = cv2.cvtColor(BEHi, cv2.COLOR_RGB2GRAY)
        #BEHi = cv2.cvtColor(BEHi, cv2.COLOR_RGB2LAB)
       
    
        #BEHi = np.float64(BEHi)
        #noise = np.random.randn(*BEHi[1].shape)*10
        #noisy = [BEHi+noise]
        #noisy = np.uint8(np.clip(BEHi,0,255))
        #dst = cv2.fastNlMeansDenoisingMulti(noisy, 2, 5, None, 4, 7, 35)
       
    if Brightness != 1 :
   
        BEHi = Image.fromarray(BEHi, 'RGB')
        BEHi = ImageEnhance.Brightness(BEHi).enhance(Brightness)
        BEHi = np.array(BEHi)
        
    
    if EnhanceEdge == True: 
        BEHi = Image.fromarray(BEHi, 'RGB')
        BEHi = BEHi.filter(ImageFilter.EDGE_ENHANCE)
        BEHi = np.array(BEHi)
        
    if EnhanceEdgeMore == True: 
        BEHi = Image.fromarray(BEHi, 'RGB')
        BEHi = BEHi.filter(ImageFilter.EDGE_ENHANCE_MORE)
        BEHi = np.array(BEHi)
        
    if substract == True:
        HandleBEHAV.set(cv2.CAP_PROP_POS_FRAMES, 0)
        _ , IMG0 = HandleBEHAV.read()
        BEHi = BEHi-IMG0
   
    if binarization != -1 :
        _ , BEHi = cv2.threshold(BEHi,binarization,255,cv2.THRESH_BINARY)
        
    if VSD == True :
        
        width,height = calculateNewSize(np.shape(VSDi)[1],np.shape(VSDi)[0],np.shape(BEHi)[1])
        VSDi = Image.fromarray(VSDi, 'RGB')
        VSDi = VSDi.resize((width,height), Image.ANTIALIAS)
        output = np.concatenate((VSDi,BEHi),axis=0)
    
    else :
        
        output = BEHi
    
    fig = plt.figure(figsize = (30,50))
    plt.imshow(output, interpolation='none')
    #[test_row.follicle_x, test_row.tip_x ]
    #[test_row.follicle_y, test_row.tip_y ]
    
    i = 0
    while i < (data.shape[0]):
       
        if data.fid[i] == time:
            plt.plot([int(data.follicle_x[i]), int(data.tip_x [i])],[int(data.follicle_y[i]), int(data.tip_y[i])], color = 'r' , linewidth = 0.8)
        i = i+1
        
        #if i > time:
         #   break
 #out = cv2.VideoWriter(video_name, cv2.VideoWriter_fourcc('M', 'J', 'P', 'G'), frame_rate, (frame_width, frame_height))
    
       
    

interactive(children=(IntSlider(value=200, description='time', max=499), FloatSlider(value=1.0, description='B…